In [58]:
import numpy as np
from pathlib import Path
from pymatreader import read_mat
from collections import Counter

import mne

%matplotlib qt


In [ ]:

# base path for biomag 2022 challenge
base_path = Path('/storage/store/data/biomag_challenge/Biomag2022/biomag_hokuto')

### Create info from .mat

In [59]:
info_path = base_path / 'training/control/control_01_25/hokuto_control10.mat'
infos = read_mat(info_path)
infos = infos['D']
print(infos.keys())

dict_keys(['type', 'Nsamples', 'Fsample', 'timeOnset', 'trials', 'channels', 'data', 'fname', 'path', 'sensors', 'fiducials', 'transform', 'condlist', 'montage'])


In [60]:
sfreq = infos['Fsample']
print(f"sfreq: {sfreq}")
print(f"infos: {infos['data']}")
data_fname = infos['data']['fname']
print(f"Data file name: {data_fname}")

sfreq: 2000
infos: {'fname': 'R:\\biomag\\training\\control\\hokuto_control10.dat', 'dim': array([   186, 600000], dtype=int32), 'dtype': 16, 'be': 0, 'offset': 0, 'pos': array([1, 1], dtype=uint8), 'scl_slope': array([], dtype=uint8), 'scl_inter': array([], dtype=uint8), 'permission': 'rw'}
Data file name: R:\biomag\training\control\hokuto_control10.dat


In [61]:
# sensor_position = data['sensors']
# fiducials = data['fiducials']
print(infos['channels'].keys())
ch_names = infos['channels']['label']
ch_types = infos['channels']['type']
print(Counter(ch_types))
# correspondance dict for channel types
type_dict = {'MEGGRAD': 'grad',
             'Other'  : 'misc',
             'REFMAG' : 'ref_meg',
             'EEG'    : 'eeg'}
ch_types = [type_dict[ch] for ch in ch_types]

dict_keys(['bad', 'label', 'type', 'X_plot2D', 'Y_plot2D', 'units'])
Counter({'MEGGRAD': 160, 'Other': 14, 'REFMAG': 12})


In [62]:
info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)
info

<Info | 7 non-empty values
 bads: []
 ch_names: AG001, AG002, AG003, AG004, AG005, AG006, AG007, AG008, AG009, ...
 chs: 160 Gradiometers, 14 misc, 12 Reference Magnetometers
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 186
 projs: []
 sfreq: 2000.0 Hz
>

### Load data from .dat and create MNE.raw object

In [63]:
#data_path = '/storage/store/data/biomag_challenge/Biomag2022/biomag_hokuto/traininig/control/control_01_25/hokuto_control10.dat'
data_path = Path(str(info_path).replace('mat', 'dat'))
data = np.fromfile(data_path, dtype="float32")  # Ca doit pas etre le bon dtype
n_ch, n_times = len(ch_types), infos['Nsamples']
data = np.reshape(data, (n_ch, n_times))

# rescale
print(Counter(infos['channels']['units']))
units = np.array(infos['channels']['units'])
data[units == 'fT'] *= 1e-15
data[units == 'uV'] *= 1e-6 
print(data)

Counter({'fT': 172, 'uV': 14})
[[-4.4437852e-12 -4.2047303e-12 -4.3959203e-12 ... -3.7867504e-12
  -2.0988523e-12 -4.5314399e-12]
 [-4.1064496e-12 -2.3293535e-12 -1.6518077e-12 ... -5.9621583e-12
  -6.7268114e-12 -2.6437424e-12]
 [-1.3329157e-12 -3.7167413e-12 -2.7221691e-12 ... -5.4609095e-12
  -3.3670506e-12 -2.7577808e-12]
 ...
 [-3.4250690e-12 -2.7168140e-12 -3.4067124e-12 ... -4.7772077e-12
  -4.3340496e-12 -4.9926734e-12]
 [-4.5256429e-12 -4.1327384e-12 -3.3934938e-12 ... -3.0314798e-12
  -2.4162246e-12 -4.6026000e-12]
 [-4.5409514e-12 -5.2908424e-12 -3.7973080e-12 ... -3.2286915e-13
  -2.4891797e-13 -3.4066993e-13]]


In [64]:
raw = mne.io.RawArray(data, info)
raw.plot(theme='light');

Creating RawArray with float64 data, n_channels=186, n_times=600000
    Range : 0 ... 599999 =      0.000 ...   300.000 secs
Ready.
Opening raw-browser...
For Dark-Mode "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Using pyopengl with version 3.1.6
Closing raw-browser...
Channels marked as bad:
none
Closing raw-browser...
Channels marked as bad:
none


TypeError: SettingsDialog.closeEvent() takes 1 positional argument but 2 were given


In [55]:
raw.plot_psd();

Effective window size : 2.048 (s)
Effective window size : 2.048 (s)


/var/folders/m5/45fck2kd7gg40pz1zs96bxlw0000gn/T/ipykernel_62655/3341777298.py:1: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  raw.plot_psd();
